In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
eil51 = pd.read_csv('51tsp.txt', delimiter=' ', index_col= 0, header = None, names = ['x','y'])

In [3]:
with open('eil51.opt.tour.txt', newline='') as file:
    lines = file.readlines()
opt51 = [int(l[:-1]) for l in lines]

In [4]:
def distance_matrix(df):
    matrix = pd.DataFrame(data = np.zeros((len(df), len(df))), index = np.arange(1,52),columns = np.arange(1,52))
    for index1, cor1 in df.iterrows():
        for index2, cor2 in df.iterrows():
            diff_x = cor1["x"] - cor2["x"]
            diff_y = cor1["y"] - cor2["y"]
            distance = (diff_x**2 + diff_y**2)**0.5
            matrix.loc[index1, index2] = distance
    return matrix

def distance(d_df, route):
    distance = 0
    for i in range(len(route)-1):
        distance += d_df.loc[route[i], route[i+1]]
    return distance

In [5]:
def insert(route):
    new_route = route[:]
    m, n = random.sample(range(0, len(new_route)), 2)
    city = new_route[m]
    new_route.remove(city)
    new_route.insert(n, city)
    return new_route
def swap(route):
    new_route = route[:]
    pos_1, pos_2 = random.sample(range(0, len(new_route)), 2)
    new_route[pos_1], new_route[pos_2] = new_route[pos_2], new_route[pos_1]
    return new_route
def reverse(route):
    copy_route = route[:]
    pos_1, pos_2 = random.sample(range(0, len(copy_route)), 2)
    if pos_2 < pos_1:
        pos_1, pos_2 = pos_2, pos_1
    reversed_ = copy_route[pos_1:pos_2][::-1]
    new_route = copy_route[0:pos_1] + reversed_ + copy_route[pos_2:]
    return new_route

In [6]:
def main_sa(d_df, route0, mutate, mc_len, sch_method, t0, stop):
    d_list = [distance(d_df, route0)]
    t = t0
#     t_list = [t]
    best_route = route0[:]

    for k in range(stop):
        for i in range(mc_len):
            new_route = mutate(best_route)
            new_distance = distance(d_df, new_route)
            diff_distance = distance(d_df, best_route) - new_distance
            d_list.append(new_distance)
            if diff_distance > 0: 
                best_route = new_route
            else:
                p = np.e**(diff_distance/t)
                if random.uniform(0, 1) < p:
                    best_route = new_route
                    
        if sch_method == "linear":
            t = t - t0/stop
        elif sch_method == "geometric":
            alpha = 0.95
            t = t * alpha
#         elif sch_method == "adaptive":

#         t_list.append(t)
        
    return d_list, best_route

In [8]:
data = eil51
d_df = distance_matrix(data)
mc_length = [10, 20]
cool_schedules = ['linear','geometric']
t0 = 100
simulations = 3
stop = 100

results = pd.DataFrame()
for cooling in cool_schedules:
    for mc_len in mc_length:
        for s in range(simulations):
            random_route = [int(x) for x in random.sample(data.index.tolist(), len(data.index))]
            d_list, best_route = main_sa(d_df, random_route, swap, mc_len, cooling, t0, stop)
            df_dict = {
              "simulation": [s],
              "cooling schedules": [cooling],
              "MC length": [mc_len],
              "found_distances": [distance(d_df, best_route)]
            }
            df = pd.DataFrame.from_dict(df_dict)
            results = results.append(df)
            print(f'{cooling} {mc_len} {s+1}/{simulations} done')
results.to_csv("test.csv", index=False)

linear101/3 done
linear102/3 done
linear103/3 done
linear201/3 done
linear202/3 done
linear203/3 done
geometric101/3 done
geometric102/3 done
geometric103/3 done
geometric201/3 done
geometric202/3 done
geometric203/3 done
